### Import Efficient 3DCNN MobileNetV2

In [1]:
import sys
sys.path.append('../Face Liveness/Models/Efficient-3DCNNs-master/models')

In [2]:
from shufflenetv2 import ShuffleNetV2

In [3]:
# Create an instance of the MobileNetV2 model
model = ShuffleNetV2(num_classes=600, sample_size=112, width_mult=1.5)

In [4]:
model.eval()

ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv3d(3, 24, kernel_size=(3, 3, 3), stride=(1, 2, 2), padding=(1, 1, 1), bias=False)
    (1): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (features): Sequential(
    (0): InvertedResidual(
      (banch1): Sequential(
        (0): Conv3d(24, 24, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), groups=24, bias=False)
        (1): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv3d(24, 88, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (3): BatchNorm3d(88, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (banch2): Sequential(
        (0): Conv3d(24, 88, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (1): BatchNorm3d(88, eps=1e-05, momentum=0.

### Load the pre-train / checkpoint

In [5]:
import torch

checkpoint_path = './kinetics_shufflenetv2_1.5x_RGB_16_best.pth'

# Load the checkpoint
checkpoint = torch.load(checkpoint_path)

C:\Users\andre\AppData\Local\Temp\ipykernel_16888\3330516624.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


In [6]:
# Remove the 'module.' prefix from the state_dict keys
new_state_dict = {}
for k, v in checkpoint['state_dict'].items():
    new_key = k.replace('module.', '')  # Remove 'module.' from key names
    new_state_dict[new_key] = v

# Load the new state_dict into your model
model.load_state_dict(new_state_dict, strict=False)

<All keys matched successfully>

In [7]:
# Print the original classifier structure
print("Original classifier:", model.classifier)

Original classifier: Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1024, out_features=600, bias=True)
)


In [8]:
# Access the last layer in the classifier to get in_features
if isinstance(model.classifier, torch.nn.Sequential):
    # Assuming the last layer is a Linear layer
    last_layer = model.classifier[-1]  # Access the last layer in the Sequential
    in_features = last_layer.in_features  # Get the number of input features
else:
    in_features = model.classifier.in_features  # If it's a Linear layer directly

print("Input features for the classifier:", in_features)

Input features for the classifier: 1024


### Replace last layer / classifier with dummy to make feature extractor

In [9]:
# Replace the classifier with a dummy layer (optional)
model.classifier = torch.nn.Identity()  # Use Identity to keep the output as features

In [10]:
# Print the new classifier structure
print("Updated classifier:", model.classifier)

Updated classifier: Identity()


In [11]:
model.eval()

ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv3d(3, 24, kernel_size=(3, 3, 3), stride=(1, 2, 2), padding=(1, 1, 1), bias=False)
    (1): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (features): Sequential(
    (0): InvertedResidual(
      (banch1): Sequential(
        (0): Conv3d(24, 24, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), groups=24, bias=False)
        (1): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv3d(24, 88, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (3): BatchNorm3d(88, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (banch2): Sequential(
        (0): Conv3d(24, 88, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (1): BatchNorm3d(88, eps=1e-05, momentum=0.

### Folders

In [12]:
train_folder = '../../Dataset/CASIA-FASD/train_release'
test_folder = '../../Dataset/CASIA-FASD/test_release'
mix_eval_folder = '../../Dataset/Mix/Validation/Face'

In [13]:
from video_dataset import VideoDataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [14]:
# train_dataset = VideoDataset(train_folder)
# train_loader = DataLoader(train_dataset, batch_size=4, shuffle=False)

In [15]:
# test_dataset = VideoDataset(test_folder)
# test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [16]:
mix_eval_dataset = VideoDataset(mix_eval_folder)
mix_eval_loader = DataLoader(mix_eval_dataset, batch_size=4, shuffle=False)

##### Show sample of the first video

In [17]:
# for frames, labels in train_loader:  # Unpack frames and labels
#     for i in range(frames.size(1)):  # frames.size(1) gives the number of frames in the first video
#         plt.imshow(frames[0][i].permute(1, 2, 0).numpy())  # Show the i-th frame of the first video
#         plt.axis('off')  # Hide axis
#         plt.title(f'Frame {i + 1} of First Video')
#         plt.show()
    
#     break  # Exit after the first batch

#### Check data shape

In [18]:
# # Check the length of the train dataset
# print("Number of videos in train dataset:", len(train_dataset))

# # Check the return value of the first item in the dataset
# sample_frames, sample_label = train_dataset[0]  # Get the first video
# print("Return value for the first video (frames, label):", (sample_frames, sample_label))

# # Check the shape of the frames
# if isinstance(sample_frames, torch.Tensor):
#     print("Shape of frames for the first video:", sample_frames.shape)
# else:
#     print("The first video frames are not a tensor, inspect further.")


### Extract Features

In [19]:
def feature_extract(loader):
    features_list, labels_list, identity_list = [], [], []
    with torch.no_grad():
        for frames, labels, identity in loader:
            batch_size = frames.size(0)
            
            # Reshape frames: [B, C, T, H, W]
            frames = frames.view(batch_size, 3, frames.size(1), 112, 112)
            
            # Normalize pixel values to [0, 1]
            frames = frames.float() / 255.0
            
            # Extract features using the model
            features = model(frames)

            features_list.append(features)
            labels_list.append(labels)
            identity_list.append(identity)

    # Concatenate the lists into tensors
    features = torch.cat(features_list, dim=0)  # Concatenate features
    labels = torch.cat(labels_list, dim=0)      # Concatenate labels
        
    return features, labels, identity_list

In [20]:
# train_features, train_labels = feature_extract(train_loader)

In [21]:
# print(train_features.shape)
# print(train_labels.shape)

In [22]:
# test_features, test_labels = feature_extract(test_loader)

In [23]:
# print(test_features.shape)
# print(test_labels.shape)

In [24]:
mix_model_features, mix_model_labels, mix_model_identities = feature_extract(mix_eval_loader)

In [25]:
mix_features_np = mix_model_features.numpy()
mix_labels_np = mix_model_labels.numpy()

In [26]:
import numpy as np

In [27]:
np.save('../../Dataset/Mix/Validation/face_features.npy', mix_features_np)
np.save('../../Dataset/Mix/Validation/face_labels.npy', mix_labels_np)
np.save('../../Dataset/Mix/Validation/face_identities.npy', mix_model_identities)

### Evaluation

In [28]:
# train_features_np = train_features.numpy()  # Convert training features to NumPy array
# train_labels_np = train_labels.numpy()      # Convert training labels to NumPy array
# test_features_np = test_features.numpy()    # Convert testing features to NumPy array
# test_labels_np = test_labels.numpy()        # Convert testing labels to NumPy array

In [29]:
# print(train_features_np.shape)
# print(train_labels_np.shape)
# print(test_features_np.shape)
# print(test_labels_np.shape)

In [30]:
# from xgboost import XGBClassifier
# from sklearn.metrics import accuracy_score, classification_report

In [31]:
# # Train an XGB classifier
# xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
# xgb_classifier.fit(train_features_np, train_labels)

# # Test the SVM classifier on the test set
# xgb_predictions = xgb_classifier.predict(test_features_np)

# # Evaluate the classifier
# accuracy = accuracy_score(test_labels, xgb_predictions)
# print(f"Accuracy: {accuracy * 100:.2f}%")

# # Print detailed classification report
# print(classification_report(test_labels, xgb_predictions))

In [32]:
# import numpy as np

In [33]:
# # Calculate the counts
# TP = np.sum((xgb_predictions == 1) & (test_labels_np == 1))  # True Positives
# TN = np.sum((xgb_predictions == 0) & (test_labels_np == 0))  # True Negatives
# FP = np.sum((xgb_predictions == 1) & (test_labels_np == 0))  # False Positives
# FN = np.sum((xgb_predictions == 0) & (test_labels_np == 1))  # False Negatives

# # Calculate FAR
# FAR = FP / (FP + TN) if (FP + TN) > 0 else 0  # Avoid division by zero

# print(f'False Acceptance Rate (FAR): {FAR * 100:.2f}%')